# Space Bandits Model as a Classifier

In [1]:
import numpy as np
import pandas as pd
from random import random, randint
import matplotlib.pyplot as plt
import gc
%config InlineBackend.figure_format='retina'

from space_bandits.toy_problem import generate_dataframe

/opt/anaconda3/envs/work/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Генерация выборки

df = generate_dataframe(20_000)
print(df.shape)
df.head()

(20000, 4)


,age,ARPU,action,reward
0,47.0,93.984068,0,0
1,30.0,80.574450,0,10
2,48.0,23.344709,2,0
3,29.0,100.416298,0,10
4,51.0,41.958373,0,0


In [3]:
df[['action','reward']].nunique()

action    3
reward    4
dtype: int64

Мы создаем набор данных со случайно выбранными действиями и 4000 строк.
## Разделение обучения/проверки
Мы разделяем данные на две группы одинакового размера.

In [4]:
train = df.sample(frac=.5).copy()
val = df[~df.index.isin(train.index)].copy()
num_actions = len(train.action.unique())

## Метрика проверки
Мы будем использовать оценку ROC AUC в качестве метрики проверки. Мы обучим простой бинарный классификатор, модель логистической регрессии, чтобы «конкурировать» с нашей моделью бандитов. Эта модель просто предсказывает конвертацию/неконвертацию.

In [5]:
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [6]:
train_fts = train[['age', 'ARPU']]

#give actions as features
campaign_fts = pd.get_dummies(train.action)
campaign_fts.index = train_fts.index
X_train = pd.concat([train_fts, campaign_fts], axis=1)

#Get labels: we are predicting conversion, so 1 if reward != 0
train['target'] = np.where(train.reward > 0, 1, 0)
y_train = train.target

#prepare X_val for later
val_fts = val[['age', 'ARPU']]
campaign_fts_val = pd.get_dummies(val.action)
campaign_fts_val.index = val_fts.index
X_val = pd.concat([val_fts, campaign_fts_val], axis=1)
#get validation labels as well
val['target'] = np.where(val.reward > 0, 1, 0)
y_val = val.target

In [7]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
pred = classifier.predict_proba(X_val)[:, 1]

classifier_auc_score = roc_auc_score(y_val, pred)
print('Logistic regression auc score: ', round(classifier_auc_score, 3))

Logistic regression auc score:  0.795


/opt/anaconda3/envs/work/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/anaconda3/envs/work/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


## Bandits Model
Мы подгоняем модель бандитов под те же данные.

In [8]:
from space_bandits import LinearBandits, NeuralBandits

model_linear = LinearBandits(num_actions=num_actions, num_features=2, initial_pulls=100)
model_neural = NeuralBandits(num_actions=num_actions, num_features=2, layer_sizes=[50,12])

In [9]:
model_linear.fit(contexts=train[['age', 'ARPU']], actions=train['action'], rewards=train['reward'])

In [10]:
model_neural.fit(contexts=train[['age', 'ARPU']], actions=train['action'], rewards=train['reward'])

Training neural_model-bnn for 100 steps...


# Получите ожидаемые вознаграждения
Мы собираем ожидаемые значения вознаграждений и добавляем их в фрейм данных проверки.

---
### 1. LinearBandits

In [11]:
expected_values = model_linear.expected_values(val[['age', 'ARPU']].values)
linear_pred = pd.DataFrame()
for a, vals in enumerate(expected_values):
    linear_pred[a] = vals
#expected reward values
linear_pred.index = val.index
#add them to validation df
linear_val = pd.concat([val, linear_pred], axis=1)
linear_val.head()

,age,ARPU,action,reward,target,0,1,2
0,47.0,93.984068,0,0,0,4.508121,2.380281,15.135053
1,30.0,80.574450,0,10,1,6.954370,2.456745,7.232880
2,48.0,23.344709,2,0,0,1.466874,2.061090,20.015796
4,51.0,41.958373,0,0,0,1.694223,2.119178,20.405841
7,52.0,39.281862,1,0,0,1.409762,2.099413,21.084911


## Применение сигмоидальной функции
Модель бандитов рассматривает каждую кампанию отдельно, поэтому мы должны применить сигмоидальную функцию к каждому столбцу вознаграждения независимо. Чтобы получить разумные значения, усредните и нормализуйте каждый ожидаемый столбец вознаграждения.

In [12]:
linear_val['pred'] = .5
for a in range(num_actions):
    #средний центр и нормализовать ожидаемые вознаграждения
    linear_val['{}_centered'.format(a)] = (linear_val[a] - linear_val[a].mean())/linear_val[a].std()

In [13]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Применить сигмоиду, чтобы получить p_pred
for a in range(num_actions):
    #получить строки для этого действия
    slc = linear_val[linear_val.action==a]
    #передавать значения через сигмоиду
    vals = sigmoid(slc['{}_centered'.format(a)].values)
    #назначить вывод соответствующим строкам
    inds = slc.index
    linear_val.loc[inds, 'pred'] = vals

In [14]:
linear_pred = linear_val.pred

bandits_auc_score = roc_auc_score(y_val, linear_pred)
print('Оценка LinearBandits roc_auc: ', round(bandits_auc_score, 3))

Оценка LinearBandits roc_auc:  0.677


---
### 2. NeuralBandits

In [15]:
expected_values = model_neural.expected_values(val[['age', 'ARPU']].values)
neural_pred = pd.DataFrame()
for a, vals in enumerate(expected_values):
    neural_pred[a] = vals
#expected reward values
neural_pred.index = val.index
#add them to validation df
neural_val = pd.concat([val, neural_pred], axis=1)
neural_val.head()

,age,ARPU,action,reward,target,0,1,2
0,47.0,93.984068,0,0,0,286.898086,121.205307,136.015916
1,30.0,80.574450,0,10,1,249.128507,95.316615,72.837542
2,48.0,23.344709,2,0,0,43.655498,41.338383,339.825304
4,51.0,41.958373,0,0,0,48.150337,53.947067,439.361253
7,52.0,39.281862,1,0,0,48.652764,52.870418,431.239338


In [16]:
neural_val['pred'] = .5
for a in range(num_actions):
    #средний центр и нормализовать ожидаемые вознаграждения
    neural_val['{}_centered'.format(a)] = (neural_val[a] - neural_val[a].mean())/neural_val[a].std()

In [17]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Применить сигмоиду, чтобы получить p_pred
for a in range(num_actions):
    #получить строки для этого действия
    slc = neural_val[neural_val.action==a]
    #передавать значения через сигмоиду
    vals = sigmoid(slc['{}_centered'.format(a)].values)
    #назначить вывод соответствующим строкам
    inds = slc.index
    neural_val.loc[inds, 'pred'] = vals

In [18]:
neural_pred = neural_val.pred

bandits_auc_score = roc_auc_score(y_val, neural_pred)
print('Оценка LinearBandits roc_auc: ', round(bandits_auc_score, 3))

Оценка LinearBandits roc_auc:  0.669


## Результат
Мы видим, что модель логистической регрессии работает лучше по этой метрике. Это не должно быть сюрпризом! У модели бандитов гораздо более сложная задача! Ей нужно выполнить регрессию для всех трех кампаний — модель логрег получает все преимущества надзора и имеет только один бинарный выход.